In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install holidays


[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import random
from scipy.stats import zscore
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import holidays

pd.set_option('display.max_columns', None)

In [3]:
def explore(df: pd.DataFrame) -> pd.DataFrame:
    ex1, ex2, ex3 = random.sample(range(len(df)), 3)

    print("Dataframe total rows: ", len(df))
    print("Dataframe total columns: ", len(df.columns))
    df_info = pd.DataFrame(data=df.dtypes)
    missing_values_total = df.notnull().sum() 
    missing_values_percent = round((1 - missing_values_total / len(df)) * 100, 2).astype(str) + " %"

    column_length = df.apply(len)  
    unique_values_count = df.nunique() 

    return pd.concat([df_info[0].rename("Data Type"),
                        df.T[ex1].rename("Example 1"),
                        df.T[ex2].rename("Example 2"),
                        df.T[ex3].rename("Example 3"),
                        column_length.rename("Column Length"),                      
                        missing_values_total.rename("Total Not Missing"),
                        missing_values_percent.rename("% of missing values"),
                        unique_values_count.rename("Unique Values Count")], axis=1)    
    
def categorical_descriptive_stats_df(dataframe):    
    categorical_cols = dataframe.columns
    columns, modas, cardinalities, unique_vals, val_counts = [], [], [], [], []

    for col in categorical_cols:
        mode = dataframe[col].mode()[0]
        cardinality = dataframe[col].nunique()
        
        columns.append(col)
        modas.append(mode)
        cardinalities.append(cardinality)
        unique_vals.append("")
        val_counts.append("")
        
        value_counts = dataframe[col].value_counts().to_dict()
        for unique_val, count in value_counts.items():
            columns.append("")
            modas.append("")
            cardinalities.append("")
            unique_vals.append(unique_val)
            val_counts.append(count)
    
    stats_df = pd.DataFrame({
        'Column': columns,
        'Moda': modas,
        'Cardinalidad': cardinalities,
        'Unique Value': unique_vals,
        'Count': val_counts
    })
    
    return stats_df.style.hide()


def find_similar_pairs(word_list, threshold=85):
    similar_pairs = []
    
    for i, word1 in enumerate(word_list):
        for j, word2 in enumerate(word_list):
            if i < j and fuzz.ratio(word1, word2) > threshold:
                similar_pairs.append((word1, word2))
                
    return similar_pairs

def get_season(date):
    if 3 <= date.month <= 5:
        return 'Spring'
    elif 6 <= date.month <= 8:
        return 'Summer'
    elif 9 <= date.month <= 11:
        return 'Fall'
    else:
        return 'Winter'


def is_holiday(row):
    year = row.year
    return row in holidays.Mexico(years=[year])

# Loading the dataset

In [4]:
df = pd.read_excel("numismatica.xlsx")
df.head()

,IDVenta,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos,Costos de envío,Total (MXN),Venta por publicidad,IDproducto,Canal de venta,Precio unitario de venta de la publicación (MXN),Municipio/Alcaldía,Estado,Código postal,Forma de entrega,Transportista,Fecha Venta,FechaCamino,FechaEntrega
0,2000003801421707,1,185.00,-83.900002,0.0,101.099998,Sí,MLM937449001,Mercado Libre,175.0,Coyoacán,Distrito Federal,4250,Mercado Envíos Full,Mercado Envios,21/06/2022,2022-06-21,2022-06-21
1,2000004169612598,1,166.25,-62.840000,-89.0,103.410004,Sí,MLM937449001,Mercado Libre,185.0,Nezahualcoyotl,Estado De México,57200,Mercado Envíos Full,Mercado Envios,02/11/2021,2021-11-02,2021-11-02
2,2000003794895775,1,185.00,-67.099998,0.0,117.900002,Sí,MLM937274388,Mercado Libre,295.0,Tultitlán,Estado De México,54933,Mercado Envíos Full,Mercado Envios,26/08/2022,2022-08-26,2022-08-26
3,2000003792874955,2,370.50,-134.320007,0.0,236.179993,Sí,MLM937274388,Mercado Libre,295.0,Gustavo A. Madero,Distrito Federal,7800,Mercado Envíos Full,Mercado Envios,12/08/2022,2022-08-12,2022-08-12
4,2000003791418641,1,185.00,-67.099998,0.0,117.900002,Sí,MLM937274388,Mercado Libre,295.0,Naucalpan,Estado De México,53580,Mercado Envíos Full,Mercado Envios,01/08/2022,2022-08-01,2022-08-01


In [5]:
explore(df)

Dataframe total rows:  2348
Dataframe total columns:  18


,Data Type,Example 1,Example 2,Example 3,Column Length,Total Not Missing,% of missing values,Unique Values Count
IDVenta,int64,2000003788299800,2000002718826985,2000002781003159,2348,2348,0.0 %,2348
Unidades,int64,1,1,3,2348,2348,0.0 %,7
Ingresos por productos (MXN),float64,157.5,145.0,465.0,2348,2348,0.0 %,157
Cargo por venta e impuestos,float64,-60.84,-58.0,-180.830002,2348,2348,0.0 %,190
Costos de envío,float64,-99.0,0.0,0.0,2348,2348,0.0 %,229
Total (MXN),float64,96.660004,87.0,284.170013,2348,2348,0.0 %,199
Venta por publicidad,object,Sí,No,No,2348,2348,0.0 %,2
IDproducto,object,MLM797897673,MLM759252135,MLM797896213,2348,2348,0.0 %,36
Canal de venta,object,Mercado Libre,Mercado Libre,Mercado Libre,2348,2348,0.0 %,2
Precio unitario de venta de la publicación (MXN),float64,155.0,155.0,199.0,2348,2348,0.0 %,88


In [6]:
# ¿Cuántas columnas númericas hay?  
numeric_columns = df.select_dtypes(include=['int64', 'float64']).shape[1]
print("Número de columnas numéricas:", numeric_columns)

# ¿Cuántas columnas cualitativas hay?
qualitative_columns = df.select_dtypes(include=['object', 'category', 'datetime64[ns]']).shape[1]
print("Número de columnas cualitativas:", qualitative_columns)

Número de columnas numéricas: 8
Número de columnas cualitativas: 10


# Duplicates

In [7]:
print("duplicate record analysis  considering Nans")
dups =  df.drop("IDVenta", axis=1)
duplicates = dups.groupby(dups.columns.tolist(), as_index=False, dropna=False).size()
duplicates[duplicates["size"]>1]

duplicate record analysis  considering Nans


,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos,Costos de envío,Total (MXN),Venta por publicidad,IDproducto,Canal de venta,Precio unitario de venta de la publicación (MXN),Municipio/Alcaldía,Estado,Código postal,Forma de entrega,Transportista,Fecha Venta,FechaCamino,FechaEntrega,size


# Fixing Dates

In [8]:
df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'])
df['FechaCamino'] = pd.to_datetime(df['FechaCamino'])
df['FechaEntrega'] = pd.to_datetime(df['FechaEntrega'])

/tmp/ipykernel_18498/3518050707.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'])


In [9]:
df.head()

,IDVenta,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos,Costos de envío,Total (MXN),Venta por publicidad,IDproducto,Canal de venta,Precio unitario de venta de la publicación (MXN),Municipio/Alcaldía,Estado,Código postal,Forma de entrega,Transportista,Fecha Venta,FechaCamino,FechaEntrega
0,2000003801421707,1,185.00,-83.900002,0.0,101.099998,Sí,MLM937449001,Mercado Libre,175.0,Coyoacán,Distrito Federal,4250,Mercado Envíos Full,Mercado Envios,2022-06-21,2022-06-21,2022-06-21
1,2000004169612598,1,166.25,-62.840000,-89.0,103.410004,Sí,MLM937449001,Mercado Libre,185.0,Nezahualcoyotl,Estado De México,57200,Mercado Envíos Full,Mercado Envios,2021-11-02,2021-11-02,2021-11-02
2,2000003794895775,1,185.00,-67.099998,0.0,117.900002,Sí,MLM937274388,Mercado Libre,295.0,Tultitlán,Estado De México,54933,Mercado Envíos Full,Mercado Envios,2022-08-26,2022-08-26,2022-08-26
3,2000003792874955,2,370.50,-134.320007,0.0,236.179993,Sí,MLM937274388,Mercado Libre,295.0,Gustavo A. Madero,Distrito Federal,7800,Mercado Envíos Full,Mercado Envios,2022-08-12,2022-08-12,2022-08-12
4,2000003791418641,1,185.00,-67.099998,0.0,117.900002,Sí,MLM937274388,Mercado Libre,295.0,Naucalpan,Estado De México,53580,Mercado Envíos Full,Mercado Envios,2022-08-01,2022-08-01,2022-08-01


# Fixing Other Columns

In [10]:
df.select_dtypes(include=['int64', 'float64']).describe()

,IDVenta,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos,Costos de envío,Total (MXN),Precio unitario de venta de la publicación (MXN),Código postal
count,2.348000e+03,2348.000000,2348.000000,2348.000000,2348.000000,2348.000000,2348.000000,2348.000000
mean,1.548556e+15,1.303663,260.704757,-91.925431,-40.250447,163.509736,206.226738,52260.152896
std,8.362938e+14,0.654208,151.946685,69.061560,47.980166,106.683288,97.738152,25702.330589
min,4.851693e+09,1.000000,85.690000,-2277.879883,-512.500000,-1402.880005,85.690002,1080.000000
25%,2.000003e+15,1.000000,166.250000,-117.029999,-94.000000,103.410004,166.250000,33057.750000
50%,2.000003e+15,1.000000,185.000000,-67.099998,0.000000,117.900002,175.000000,54040.000000
75%,2.000004e+15,1.000000,315.000000,-62.840000,0.000000,202.869995,185.000000,73941.000000
max,2.000004e+15,8.000000,2475.000000,-11.180000,0.000000,1399.219971,887.250000,99904.000000


In [11]:
# No encontramos errores en los datos numéricos

In [12]:
categoric_df = df.select_dtypes(include=['object', 'category'])
categorical_descriptive_stats_df(categoric_df)

Column,Moda,Cardinalidad,Unique Value,Count
Venta por publicidad,No,2,,
,,,No,1628
,,,Sí,720
IDproducto,MLM797896213,36,,
,,,MLM797896213,888
,,,MLM775876787,430
,,,MLM797897673,397
,,,MLM937274388,185
,,,MLM759252135,105
,,,MLM803071179,64


No encontramos errores a simple vista (en las variables chiquitas), así que habrá que revisar sí hay Estados o Municipios mal escritos

In [13]:
states =  df["Estado"].unique()
find_similar_pairs(states)

[('Baja California', 'Baja California Sur')]

In [14]:
# Aquí arreglamos 
correction_dict = {"Baja California": "Baja California Norte"}
df['Estado'] = df['Estado'].replace(correction_dict)

In [15]:
municipios = df["Municipio/Alcaldía"].unique()
find_similar_pairs(municipios)

[('Cuauhtémoc', 'Cuauhtemoc'),
 ('Benito juárez', 'Benito Juárez'),
 ('Venustiano Carranza', 'Venustiano\xa0Carranza'),
 ('Torreón', 'Torreon'),
 ('Acapulco', 'Aculco'),
 ('San Luis Potosí', 'San\xa0Luis\xa0Potosí'),
 ('Zacatepec', 'Zinacantepec'),
 ('Tula De Allende', 'Talpa De Allende'),
 ('Cihuatlan', 'Chumatlan')]

In [16]:
# Aquí arreglamos lo que se vea necesario (la selección es manual)
correction_dict = {"Cuauhtemoc": "Cuauhtémoc",
                   'Benito juárez': 'Benito Juárez',
                    'Venustiano\xa0Carranza': 'Venustiano Carranza',
                    'Torreon':'Torreón', 
                    'San\xa0Luis\xa0Potosí':'San Luis Potosí'}
df['Municipio/Alcaldía'] = df['Municipio/Alcaldía'].replace(correction_dict)

# Feature Engineering for the KPIS
- KPI: Venta (Cantidad)
- KPI: Ganancia ($)
- KPI: Tiempo promedio de entrega


In [17]:
df['Ganancia'] = df['Total (MXN)'] - df['Costos de envío'] - df['Cargo por venta e impuestos']
df["Tiempo de entrega (días)"] = (df["FechaEntrega"] - df["Fecha Venta"]).dt.days

In [18]:
df.sort_values("Tiempo de entrega (días)", ascending=False).head()

,IDVenta,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos,Costos de envío,Total (MXN),Venta por publicidad,IDproducto,Canal de venta,Precio unitario de venta de la publicación (MXN),Municipio/Alcaldía,Estado,Código postal,Forma de entrega,Transportista,Fecha Venta,FechaCamino,FechaEntrega,Ganancia,Tiempo de entrega (días)
2347,4974861814,1,169.00,-63.470001,-94.0,105.529999,No,MLM775876787,Mercado Libre,169.000000,Ensenada,Baja California Norte,22800,Mercado Envíos Full,DHL,2021-12-03,2021-12-04,2021-12-15,263.000000,12
2346,2000002787535296,1,295.00,-92.129997,0.0,202.869995,No,MLM797896213,Mercado Libre,175.000000,Naucalpan,Estado De México,53300,Mercado Envíos Full,Mercado Envios,2021-11-25,2021-12-04,2021-12-06,294.999992,11
2345,2000002750295123,2,358.00,-131.479996,0.0,226.520004,No,MLM775876787,Mercado Libre,169.000000,Acultzingo,Veracruz,94760,Mercado Envíos Full,DHL,2021-12-04,2021-12-04,2021-12-14,358.000000,10
2344,2000002792989094,1,162.75,-62.029999,0.0,100.720001,No,MLM797896213,Mercado Libre,166.250000,Chihuahua,Chihuahua,31310,Mercado Envíos Full,Mercado Envios,2021-12-17,2021-12-18,2021-12-27,162.750000,10
2343,2000003763737334,1,169.00,-63.470001,-94.0,105.529999,No,MLM797897673,Mercado Libre,142.600006,Palmar De Bravo,Puebla,75500,Mercado Envíos Full,FedEx,2022-05-23,2022-05-24,2022-06-02,263.000000,10


In [19]:


for column_date in df.select_dtypes(include=['datetime64[ns]']).columns:
    df[f'{column_date}_season'] = df[column_date].apply(get_season)

    # Day of the week
    df[f'{column_date}_day_of_week'] = df[column_date].dt.day_name()

    # Holiday
    df[f'{column_date}_holiday'] = df[column_date].apply(is_holiday)


    # Month
    df[f'{column_date}_month'] = df[column_date].dt.month_name()

    # Year
    df[f'{column_date}_year'] = df[column_date].dt.year



In [20]:
df.head()

,IDVenta,Unidades,Ingresos por productos (MXN),Cargo por venta e impuestos,Costos de envío,Total (MXN),Venta por publicidad,IDproducto,Canal de venta,Precio unitario de venta de la publicación (MXN),Municipio/Alcaldía,Estado,Código postal,Forma de entrega,Transportista,Fecha Venta,FechaCamino,FechaEntrega,Ganancia,Tiempo de entrega (días),Fecha Venta_season,Fecha Venta_day_of_week,Fecha Venta_holiday,Fecha Venta_month,Fecha Venta_year,FechaCamino_season,FechaCamino_day_of_week,FechaCamino_holiday,FechaCamino_month,FechaCamino_year,FechaEntrega_season,FechaEntrega_day_of_week,FechaEntrega_holiday,FechaEntrega_month,FechaEntrega_year
0,2000003801421707,1,185.00,-83.900002,0.0,101.099998,Sí,MLM937449001,Mercado Libre,175.0,Coyoacán,Distrito Federal,4250,Mercado Envíos Full,Mercado Envios,2022-06-21,2022-06-21,2022-06-21,185.000000,0,Summer,Tuesday,False,June,2022,Summer,Tuesday,False,June,2022,Summer,Tuesday,False,June,2022
1,2000004169612598,1,166.25,-62.840000,-89.0,103.410004,Sí,MLM937449001,Mercado Libre,185.0,Nezahualcoyotl,Estado De México,57200,Mercado Envíos Full,Mercado Envios,2021-11-02,2021-11-02,2021-11-02,255.250004,0,Fall,Tuesday,False,November,2021,Fall,Tuesday,False,November,2021,Fall,Tuesday,False,November,2021
2,2000003794895775,1,185.00,-67.099998,0.0,117.900002,Sí,MLM937274388,Mercado Libre,295.0,Tultitlán,Estado De México,54933,Mercado Envíos Full,Mercado Envios,2022-08-26,2022-08-26,2022-08-26,185.000000,0,Summer,Friday,False,August,2022,Summer,Friday,False,August,2022,Summer,Friday,False,August,2022
3,2000003792874955,2,370.50,-134.320007,0.0,236.179993,Sí,MLM937274388,Mercado Libre,295.0,Gustavo A. Madero,Distrito Federal,7800,Mercado Envíos Full,Mercado Envios,2022-08-12,2022-08-12,2022-08-12,370.500000,0,Summer,Friday,False,August,2022,Summer,Friday,False,August,2022,Summer,Friday,False,August,2022
4,2000003791418641,1,185.00,-67.099998,0.0,117.900002,Sí,MLM937274388,Mercado Libre,295.0,Naucalpan,Estado De México,53580,Mercado Envíos Full,Mercado Envios,2022-08-01,2022-08-01,2022-08-01,185.000000,0,Summer,Monday,False,August,2022,Summer,Monday,False,August,2022,Summer,Monday,False,August,2022


In [21]:
df.to_csv('numismatica_cleaned.csv', index=False)